Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menu bar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menu bar, select Cell$\rightarrow$Run All).

Below, please fill in your name and collaborators:

# <font color='Blue'> Group 5 Assignment - Predict Future Sales </font>


In [2]:
NAME = "Group 5"
COLLABORATORS = "Adejobi, Temidayo ; Koffi, Beniamkem ; Kovtun, Oksana ; Luo, Haiming ; Parra, Karen ;Vaithianathan, Elanchezhian"

## <font color='Blue'>Assignment tasks</font>

Group assignment objectivie is to showcase what is learned in the course. Group shall able to choose which type of analysis they would like to complete from the options below.

Choose one of the following for your project:

1. Make a hypothesis about a correlation in a dataset and test the hypothesis using a statistical inference technique (such as the t-test).
2. Build a predictive model using one of the techniques covered in the course, i.e., ordinary least squares regression or Naïve Bayes.

<font color = 'Blue'>The dataset chosen on which the analysis is "Predict Future Sales" dataset.</font>    

The dataset was downloaded from the public dataset on Kaggle at the url, https://www.kaggle.com/c/competitive-data-science-predict-future-sales/data

The dataset is being used under the terms of the licence below.

License: This work is licensed under the Creative Commons Attribution 3.0 Unported License. To view a copy of this license, visit http://creativecommons.org/licenses/by/3.0/.

# <font color='Blue'>1. Data preparation</font>


In [3]:
# import libs
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import seaborn as sns
from scipy import stats
from statsmodels.tools.eval_measures import rmse

%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

Brief note of the dataset is provided below. Predict future sales dataset cons assignment objectivie is to showcase what is learned in the course. Group shall able to choose which type of analysis they would like to complete from the options below.

Choose one of the following for your project:

1. sales_train_v2.csv - Main dataset that provides the sales data per month and would be use to train the model.
2. shops.csv - Shop id to shop name
3. item_categories.csv - Item category id to Item category name
4. items.csv - Item id to Item name.
5. test.csv - dataset that would be used to test the model.

## 1.1 Loading the dataset

In [4]:
# Read dataset file that was donwloaded
#sales_dirty_df = pd.read_csv(filepath_or_buffer='sales_train_v2.csv', sep=',')
sales_train_df = pd.read_csv('sales_train_v2.csv', sep=',',header=0)
shops_df = pd.read_csv('shops.csv', sep=',',header=0)
item_category_df = pd.read_csv('item_categories.csv', sep=',',header=0)
items_df = pd.read_csv('items.csv', sep=',',header=0)
test_df = pd.read_csv('test.csv', sep=',',header=0)


## 1.2 Analyse the dataset 'sales_train_v2.csv'

### 1.2.1 Basic inspection of the original dataset

In [5]:
# View info()
sales_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
date              object
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [6]:
sales_train_df.dropna(inplace=True)

In [7]:
# view tail of data# view head 
sales_train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [8]:
# view tail of data
sales_train_df.tail()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2935844,10.10.2015,33,25,7409,299.0,1.0
2935845,09.10.2015,33,25,7460,299.0,1.0
2935846,14.10.2015,33,25,7459,349.0,1.0
2935847,22.10.2015,33,25,7440,299.0,1.0
2935848,03.10.2015,33,25,7460,299.0,1.0


### 1.2.2 Basic inspection of the supporting dataset

In [9]:
# Analyzing items dataset
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
item_name           22170 non-null object
item_id             22170 non-null int64
item_category_id    22170 non-null int64
dtypes: int64(2), object(1)
memory usage: 519.7+ KB


In [10]:
# Analyzing Shops dataset
shops_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
shop_name    60 non-null object
shop_id      60 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.0+ KB


In [11]:
# Analyzing Items category dataset
item_category_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
item_category_name    84 non-null object
item_category_id      84 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.4+ KB


In [12]:
# Analyzing test dataset
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
ID         214200 non-null int64
shop_id    214200 non-null int64
item_id    214200 non-null int64
dtypes: int64(3)
memory usage: 4.9 MB


### 1.3 Data cleansing and transformation

In [13]:
# Split the date into day, month and year to facilitate deeper interpretation.

raw_date = sales_train_df.date.apply(lambda x:datetime.datetime.strptime(x, '%d.%m.%Y'))

sales_train_df['day'] = raw_date.dt.day
sales_train_df['month'] = raw_date.dt.month
sales_train_df['year'] = raw_date.dt.year

sales_train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year
0,02.01.2013,0,59,22154,999.00,1.0,2,1,2013
1,03.01.2013,0,25,2552,899.00,1.0,3,1,2013
2,05.01.2013,0,25,2552,899.00,-1.0,5,1,2013
3,06.01.2013,0,25,2554,1709.05,1.0,6,1,2013
4,15.01.2013,0,25,2555,1099.00,1.0,15,1,2013


In [14]:
# calculate the sales store per month
sales_train_df['sales']=sales_train_df.item_price*sales_train_df.item_cnt_day

In [15]:
# Add item category to the sales train dataset
sales_train_df = pd.merge(sales_train_df,items_df, on='item_id')
sales_train_df.drop('item_name', axis=1,inplace=True)

In [16]:
sales_train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,sales,item_category_id
0,02.01.2013,0,59,22154,999.0,1.0,2,1,2013,999.0,37
1,23.01.2013,0,24,22154,999.0,1.0,23,1,2013,999.0,37
2,20.01.2013,0,27,22154,999.0,1.0,20,1,2013,999.0,37
3,02.01.2013,0,25,22154,999.0,1.0,2,1,2013,999.0,37
4,03.01.2013,0,25,22154,999.0,1.0,3,1,2013,999.0,37


# <font color='Blue'>2. Feature Analysis: </font>